In [1]:
#!pip install gridworld

In [8]:
from gridworld import gridworld
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def standard_grid():
    """
    Cria uma grid padrão.
    """
    grid = gridworld(3,4,[(1,1)],[(0,3),(1,3)],(2,0),{(0,3): 1,(1,3): -1})
    return grid


class gridworld:
    def __init__(self, rows, columns, walls, end_states, start_position, rewards, allowed_actions=['U', 'D', 'L', 'R']):
        
        self.grid = np.ones((rows,columns))
        self.max_rows = rows
        self.max_cols = columns
        
        for wall in walls:
            self.grid[wall[0],wall[1]] = 0
        
        self.end_states = end_states
        self.current_position_row = start_position[0]
        self.current_position_col = start_position[1]
        self.rewards = rewards
        self.allowed_actions = allowed_actions
        self.all_positions = self.possible_states()
        self.start_position = start_position
        
    
    def change_position(self,d_rows, d_cols):
        """
        Move o agente na grid de acordo com a ação dada.

        Parâmetros:
            d_rows (int): Ação na direção das linhas .
            d_cols (int): Ação na direção das colunas (esquerda: -1, direita: 1).

        A função calcula a nova posição do agente adicionando os valores de d_rows e d_cols às
        coordenadas da posição atual do agente. Em seguida, verifica se a nova posição é permitida na grid,
        com o método is_position_allowed(). Se a nova posição for válida, atualiza a posição
        atual do agente.
        """
        target_position_row = self.current_position_row + d_rows
        target_position_col = self.current_position_col + d_cols
        
        check_position = self.is_position_allowed((target_position_row,target_position_col))
        if check_position:
            self.current_position_row = target_position_row
            self.current_position_col = target_position_col

        
    def check_if_game_over(self):
        """
        Confirma se a posição do agente está nas posições definidas como estados finais.
        
        Return: 
            'True' se a posição do agente estiver num estado final.
            'False' caso a posição do agente não esteja num estado final.
        """
        if (self.current_position_row, self.current_position_col) in self.end_states:
            return True
        else:
            return False
    
    def move(self, action, epsilon = 0.05):
        """
        Move o agente na grid de acordo com a ação dada.
        
        Parâmetros:
        action: Ação a ser executada pelo agente.
        epsilon: Probabilidade de escolher uma ação aleatória        
        
        """
        d_rows = 0
        d_cols = 0
        
        if np.random.rand() < epsilon: 
            action = np.random.choice(self.allowed_actions)
            
        if action in self.allowed_actions:
            
            if action == 'U':
                d_rows = -1
            elif action == 'D':
                d_rows = 1
            elif action == 'L':
                d_cols = -1
            elif action == 'R':
                d_cols = 1
                
            self.change_position(d_rows, d_cols)
    
    def possible_states(self):
        """
        Retorna uma lista com todas as opções possíveis na grid onde o agente pode estar.
        
        Return: 
        lista com todas as posições possíveis na grid.
        """
        
        output = []
        for row in range(self.max_rows):
            for col in range(self.max_cols):
                if self.grid[row,col]:
                    output.append((row,col))
        return output
    
    def is_position_allowed(self, position):
        """Verifica se uma posição na grid é permitida.
        
        Parâmetro:
        position: as coordenadas (linha, coluna) da posição que se pretende verificar.
        
        Return:
        'True' se a ação for permitida.
        'False' se a ação não for permitida.
        """
        
        p_row = position[0]
        p_col = position[1]

        if (p_row >= 0) & (p_row < self.max_rows):
            if (p_col >= 0) & (p_col < self.max_cols):
                if self.grid[p_row,p_col]:
                    return True
        return False
    
    def new_position(self, new_position):
        """
        Atualiza a posição atual do agente para nova posição.
        
        Parâmetros:
        new_position: As coordenadas (linha, coluna) da nova posição.
        """
        if self.grid[new_position[0],new_position[1]]:
            self.current_position_row = new_position[0]
            self.current_position_col = new_position[1]
    
    def current_position(self):
        """
        Retorna a posição atual do agente. 
        
        Return:
        As coordenadas (linha,coluna) da posição atual do agente.
        """
        return (self.current_position_row, self.current_position_col)
        
    
    def what_if_move(self, start, action):
        """
        Calcula a posição seguinte se o agente realizar determinada ação.
        
        Parâmetros:
        start: as coordenadas da posição atual do agente.
        action: A ação a ser considerada.
        
        Return:
        As coordenadas (linha, coluna) após o agente realizar uma ação permitida. 
        Se a ação obtiver uma coordenada que não é permitida, o agente retorna à posição inicial.
        """
        
        start_row = start[0]
        start_col = start[1]
        
        d_rows = 0
        d_cols = 0

        if action in self.allowed_actions:
            if action == 'U':
                d_rows = -1
            elif action == 'D':
                d_rows = 1
            elif action == 'L':
                d_cols = -1
            elif action == 'R':
                d_cols = 1
        
        target_position_row = start_row + d_rows
        target_position_col = start_col + d_cols
        
        target_position = (target_position_row, target_position_col)
        
        check_position = self.is_position_allowed(target_position)
        
        if check_position:
            return target_position
        else:
            return start

In [13]:
def print_politica(P,grid):
    """
    Faz print a uma politica P.
    
    Parâmetros:
    P: Posições na grid para determinadas ações.
    grid: o grid onde a política está a ser aplicada.
    
    A função faz um ciclo for para cada linha na grid e imprime a ação associada.
    Se a linha não estiver presente em P, imprime um espaço em branco.
    
    """
    rows, cols = grid.grid.shape
    for row in range(rows):
        print ("----------------")
        for col in range(cols):
            a = P.get((row,col),' ')
            print(f' {a} |', end="")
        print("")
    
def print_grid(grid):
    """
    Faz print a uma grid em que cada posição consiste
    nas coordenadas na matriz. 
    """
    rows, cols = grid.grid.shape
    for row in range(rows):
        print ("--------------------------------")
        for col in range(cols):
            print(f' ({row},{col}) |', end = "")
        
        print("")

def print_value(V, grid):
    """
    Faz print à função valor V.
    """
    rows, cols = grid.grid.shape
    for row in range(rows):
        print ("----------------------------")
        for col in range(cols):
            v = V.get((row,col),0)
            a = np.round(np.abs(v),2)
            prefix = "-" if v < 0 else " "
            print(f'{prefix}%.2f |' % a, end="")

        print("")

policy = {
     (0, 0): 'R',
     (0, 1): 'R',
     (0, 2): 'R',
     (1, 0): 'U',
     (1, 2): 'R',
     (2, 0): 'U',
     (2, 1): 'R',
     (2, 2): 'U',
     (2, 3): 'L'
}

V = {
     (0, 0): -1,
     (0, 1): 2,
     (0, 2): 4,
     (1, 0): 7,
     (1, 2): 0.2,
     (2, 0): 4,
     (2, 1): 1,
     (2, 2): 9,
     (2, 3): 0.5
}

print_politica(policy, grid)

print_grid(grid)

print_value(V, grid)

def politica_random(grid):
    """Cria uma política aleatória.
    
    *Parâmetros:
    grid: o grid onde a política está a ser aplicada.
    
    Return:
    Um dicionário que mapeia posições na grid para ações aleatórias.
    
    A função cria uma política aleatória para a grid estabelecida. 
    Para cada posição que não esteja comtemplada como estado final, uma ação aleatória é escolhida.
    A política resultante é um dicionário onde as keys são as posições na grid e os values são as ações associadas.
    """
    states = grid.all_positions
    end_states = grid.end_states
    all_actions = grid.allowed_actions
    P = {}
    
    # Loop random choice
    for state in states:
        if state not in end_states:
            P[state] = np.random.choice(all_actions)
    
    return P

def epsilon_action(a, eps=0.1, grid=grid):
    """
    Escolhe a acção a com uma probabilidade 1-eps e uma acção random 
    com uma probabilidade eps.
    """
    p = np.random.random()
    if p < (1 - eps):
        return a
    else:
        return np.random.choice(grid.allowed_actions)


def td_game(grid, policy):
    """Faz a simulação de um episódio completo do jogo usando a política dada.
    
    Parâmetros:
    grid: o grid onde a política está a ser aplicada.
    policy: política que define as ações a serem tomadas.
    
    Return:
    Uma lista com os estados visitados e as respetivas recompensas.
    
    A função simula um episódio completo de um jogo utiliznado a política dada.
    Começa na posição inicial e continua tendo em consideração as ações de acordo com a política.
    Pára quando o agente se encontrar num estado final.
    """
    s = grid.start_position
    grid.new_position(s)
    states_and_rewards = [(s, 0)] 
    while not grid.check_if_game_over():
        a = policy[s]
        a = epsilon_action(a)
        grid.move(a)
        s = grid.current_position()
        r = grid.rewards.get(s,0)
        states_and_rewards.append((s, r))
    return states_and_rewards

gamma = 0.9
lr = 0.1

grid = standard_grid()

# politica para avaliar
policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'R',
    (0, 1): 'R',
    (0, 2): 'R',
    (1, 2): 'U',
    (2, 1): 'R',
    (2, 2): 'U',
    (2, 3): 'L',
}

V = {}
states = grid.all_positions
for s in states:
    V[s] = 0

for i in range(100000):

    states_and_rewards = td_game(grid, policy)
    for t in range(len(states_and_rewards) - 1):
        s, _ = states_and_rewards[t]
        s2, r = states_and_rewards[t+1]
        V[s] = V[s] + lr*(r + gamma*V[s2] - V[s])

print("values:")
print_value(V, grid)
print("policy:")
print_politica(policy, grid)

----------------
 R | R | R |   |
----------------
 U |   | R |   |
----------------
 U | R | U | L |
--------------------------------
 (0,0) | (0,1) | (0,2) | (0,3) |
--------------------------------
 (1,0) | (1,1) | (1,2) | (1,3) |
--------------------------------
 (2,0) | (2,1) | (2,2) | (2,3) |
----------------------------
-1.00 | 2.00 | 4.00 | 0.00 |
----------------------------
 7.00 | 0.00 | 0.20 | 0.00 |
----------------------------
 4.00 | 1.00 | 9.00 | 0.50 |
values:
----------------------------
 0.76 | 0.85 | 0.98 | 0.00 |
----------------------------
 0.68 | 0.00 | 0.76 | 0.00 |
----------------------------
 0.60 | 0.59 | 0.63 | 0.57 |
policy:
----------------
 R | R | R |   |
----------------
 U |   | U |   |
----------------
 U | R | U | L |
